In [1]:
import sys
sys.path.append(r'..\scripts')

import numpy as np

## Kernel

### Compute Kernel if not cached

In [ ]:
from compute_kernels import parallel_Li

In [ ]:
#Define simulation parameters

N = 120 # Number of spatial points
zb, zt = 1, 5 #bounds
k = 2 #wave number
alpha = 4.8
dz = (zt - zb)/N

#Radius function
R = lambda z : 1
R = np.vectorize(R)

if __name__ == '__main__':
    zg, oc, L = parallel_Li(zb, zt, N, k, R, num_workers = None)

In [ ]:
np.savez(f'../data_cache/discretizationN={N}.npz', zg=zg, oc=oc, L=L)

### Load Cached Kernel

In [2]:
#Define simulation parameters

N = 120 # Number of spatial points
zb, zt = 1, 5 #bounds
k = 2 #wave number
alpha = 4.8
dz = (zt - zb)/N

#Radius function
R = lambda z : 1
R = np.vectorize(R)

discretization = np.load(f'../data_cache/discretizationN={N}.npz')
zg, oc, Li = discretization['zg'], discretization['oc'], discretization['L']

## Compute Steady States

In [ ]:
from steady_state_analysis import compute_adomian_decomp_terms, pseudo_arclength_continutation, compute_steady_state
from simulation import simulate
from dynamical_system import F_k
import matplotlib.pyplot as plt

### Pseudo-Arclength

In [ ]:
#Default Parameters
T_final = 300
gamma = 0.1
S_hat = 100
delta_hat = 0.1
kappa = 30
Delta0_func = lambda z : 1
eta0_func = lambda z : 1 + 0j
Delta_E = np.vectorize(lambda z : 0.5)

sol_T, sol_Delta, sol_eta = simulate(kappa, gamma, S_hat, delta_hat, Delta0_func, eta0_func, Delta_E, R, k, zb, zt, N, T_final, Li, oc, solver='BDF')

In [ ]:
#Initial guess for state variable is 
z0 = np.concatenate([sol_Delta[:, -1], sol_eta[:, -1]]) # Initial guess for state variables
lambda0 = np.array([100])  # Initial guess for parameter vector (3 components)
zlambd0 = np.concatenate([z0, lambda0])

# Parameters for continuation
s0 = 0.0  # Initial arclength
ds = 2  # Arclength increment
num_steps = 100  # Number of continuation steps

dz = (zt - zb)/N

function_params = (L, oc, Delta_E(zg), F_k(R, zg, k)) #gamma, L, oc, DeltaE, Fk, N, dz 

# Perform continuation
results, convergence = pseudo_arclength_continuation(zlambd0, function_params, N, dz, ds, S_hat = S_hat, delta_hat = delta_hat, kappa = kappa, gamma=gamma, max_steps=num_steps, jac_epislon=1e-8)

np.savez(f'SteadyStatesVaringKappaPseudoArc.npz', results = results, convergence=convergence) 

results
